In [4]:
# Classify the speakers of given features (self-attention)
# 使用transformer(本质上还是self-attention)解决时序问题
# 数据集 https://www.robots.ox.ac.uk/~vgg/data/voxceleb/

In [5]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

In [6]:
class myDataset(Dataset):
  def __init__(self, data_dir, segment_len=128):
    # segment_len = 128 表示时间序列上，取长度为128的时间
    # 同时一个时间片上的向量表示维度是40维
    self.data_dir = data_dir
    self.segment_len = segment_len

    # Load the mapping from speaker neme to their corresponding id.
    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())
    self.speaker2id = mapping["speaker2id"]

    # Load metadata of training data.
    metadata_path = Path(data_dir) / "metadata.json"
    metadata = json.load(open(metadata_path))["speakers"]

    # Get the total number of speaker.
    self.speaker_num = len(metadata.keys())
    self.data = []
    for speaker in metadata.keys():
      for utterances in metadata[speaker]:
        self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
              # data中存的是[文件名,speaker编号]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    feat_path, speaker = self.data[index]
    # Load preprocessed mel-spectrogram.
    mel = torch.load(os.path.join(self.data_dir, feat_path))

    # Segmemt mel-spectrogram into "segment_len" frames.
    if len(mel) > self.segment_len:
      # Randomly get the starting point of the segment.
      start = random.randint(0, len(mel) - self.segment_len)
      # Get a segment with "segment_len" frames.
      mel = torch.FloatTensor(mel[start:start+self.segment_len])
      # 如果长度大于阈值 随机选取长度为阈值的片段
    else:
      mel = torch.FloatTensor(mel)
      # 长度小于等于阈值 全部取出
    # Turn the speaker id into long for computing loss later.
    speaker = torch.FloatTensor([speaker]).long()
    return mel, speaker

  def get_speaker_number(self):
    return self.speaker_num

In [7]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
  # Process features within a batch.
  """Collate a batch of data."""
  mel, speaker = zip(*batch)
  # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
  mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
  # pad_sequence将mel中的n个不同长度的sequence在新的一维拼接起来，变成batch_size=n batch内每个元素长度为max_len
  # padding_value设定padding的时候用什么值填充
  # mel: (batch size, length, 40)
  return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
  """Generate dataloader"""
  dataset = myDataset(data_dir)
  speaker_num = dataset.get_speaker_number()
  # Split dataset into training dataset and validation dataset
  trainlen = int(0.9 * len(dataset))
  lengths = [trainlen, len(dataset) - trainlen]
  trainset, validset = random_split(dataset, lengths)

  train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  valid_loader = DataLoader(
    validset,
    batch_size=batch_size,
    num_workers=n_workers,
    drop_last=True,
    pin_memory=True,
    collate_fn=collate_batch,
  )

  return train_loader, valid_loader, speaker_num

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Classifier(nn.Module):
  def __init__(self, d_model=80, n_spks=600, dropout=0.1):
    super().__init__()
    # Project the dimension of features from that of input into d_model.
    self.prenet = nn.Linear(40, d_model)
    # TODO:
    #   Change Transformer to Conformer.
    #   https://arxiv.org/abs/2005.08100
    self.encoder_layer = nn.TransformerEncoderLayer(
      d_model=d_model, dim_feedforward=256, nhead=2
    )
    # nhead并不一定越大越好
    # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

    # Project the the dimension of features from d_model into speaker nums.
    self.pred_layer = nn.Sequential(
      # nn.Linear(d_model, d_model),
      # nn.ReLU(),
      nn.Linear(d_model, n_spks),
    )
      # 全连接层并不是越多越好

  def forward(self, mels):
    """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
    # out: (batch size, length, d_model)
    out = self.prenet(mels)
    # out: (length, batch size, d_model)
    out = out.permute(1, 0, 2)
    # The encoder layer expect features in the shape of (length, batch size, d_model).
    # the output of encoder is (length, batch_size, d_model)
    out = self.encoder_layer(out)
    # out: (batch size, length, d_model)
    out = out.transpose(0, 1)
    # mean pooling out:(batch_size, d_model) mean将length维做平均去掉
    stats = out.mean(dim=1)

    # out: (batch, n_spks)
    out = self.pred_layer(stats)
    return out

In [9]:
# Learning rate schedule

# For transformer architecture, the design of learning rate schedule is different from that of CNN.
# Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
# The warmup schedule
  # Set learning rate to 0 in the beginning.
  # The learning rate increases linearly from 0 to initial learning rate during warmup period.

# 使用warmup 进行learning rate的调整

In [10]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
  optimizer: Optimizer,
  num_warmup_steps: int,    # warmup的步数
  num_training_steps: int,  # 训练的总步数
  num_cycles: float = 0.5,  # cos函数的波数 默认0.5
  last_epoch: int = -1,     # 恢复训练时最后一个epoch的索引
):
  """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """

  def lr_lambda(current_step):
    # Warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
      # 随着current_step增加 lr不断增加
    # decadence
    progress = float(current_step - num_warmup_steps) / float(
      max(1, num_training_steps - num_warmup_steps)
    )
    # 随着current_step增加 progress增加 范围取值为[0,1]
    return max(
      0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
           # 随着progress增加，cos内部值增加 整体减少 所以lr是衰减阶段
    )

  return LambdaLR(optimizer, lr_lambda, last_epoch)

In [11]:
def model_fn(batch, model, criterion, device):
  """Forward a batch through the model."""

  mels, labels = batch
  mels = mels.to(device)
  labels = labels.to(device)

  outs = model(mels)

  loss = criterion(outs, labels)

  # Get the speaker id with highest probability.
  preds = outs.argmax(1)
  # Compute accuracy.
  accuracy = torch.mean((preds == labels).float())

  return loss, accuracy

In [12]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device):
  """Validate on validation set."""

  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      loss, accuracy = model_fn(batch, model, criterion, device)
      running_loss += loss.item()
      running_accuracy += accuracy.item()

    pbar.update(dataloader.batch_size)
    pbar.set_postfix(
      loss=f"{running_loss / (i+1):.2f}",
      accuracy=f"{running_accuracy / (i+1):.2f}",
    )

  pbar.close()
  model.train()

  return running_accuracy / len(dataloader)

In [15]:

from torch.optim import AdamW

def parse_args():
  """arguments"""
  config = {
    "data_dir": "../../Dataset_for_chapter4",
    "save_path": "model.ckpt",
    "batch_size": 128,
    "n_workers": 8,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 10000,
    "total_steps": 70000,
  }

  return config


def main(
  data_dir,
  save_path,
  batch_size,
  n_workers,
  valid_steps,
  warmup_steps,
  total_steps,
  save_steps,
):
  """Main function."""
  device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  train_iterator = iter(train_loader)
  print(f"[Info]: Finish loading data!",flush = True)

  model = Classifier(n_spks=speaker_num).to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = AdamW(model.parameters(), lr=1e-3)
  scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
  print(f"[Info]: Finish creating model!",flush = True)

  best_accuracy = -1.0
  best_state_dict = None

  pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

  for step in range(total_steps):
    # Get data
    try:
      batch = next(train_iterator)
    except StopIteration:
      train_iterator = iter(train_loader)
      batch = next(train_iterator)

    loss, accuracy = model_fn(batch, model, criterion, device)
    batch_loss = loss.item()
    batch_accuracy = accuracy.item()

    # Updata model
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

    # Log
    pbar.update()
    pbar.set_postfix(
      loss=f"{batch_loss:.2f}",
      accuracy=f"{batch_accuracy:.2f}",
      step=step + 1,
    )

    # Do validation
    if (step + 1) % valid_steps == 0:
      pbar.close()

      valid_accuracy = valid(valid_loader, model, criterion, device)

      # keep the best model
      if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_state_dict = model.state_dict()

      pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    # Save the best model so far.
    if (step + 1) % save_steps == 0 and best_state_dict is not None:
      torch.save(best_state_dict, save_path)
      pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

  pbar.close()


if __name__ == "__main__":
  main(**parse_args())

Train:   5% 105/2000 [03:31<1:03:38,  2.01s/ step, accuracy=0.00, loss=6.33, step=105]

[Info]: Use cuda:3 now!


[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [04:16<00:00,  7.79 step/s, accuracy=0.35, loss=3.10, step=2000] 
Valid: 100% 6912/6944 [00:27<00:00, 254.98 uttr/s, accuracy=0.34, loss=3.05]
Train: 100% 2000/2000 [00:22<00:00, 87.00 step/s, accuracy=0.50, loss=2.15, step=4000] 
Valid: 100% 6912/6944 [00:00<00:00, 10737.12 uttr/s, accuracy=0.46, loss=2.43]
Train: 100% 2000/2000 [00:22<00:00, 89.24 step/s, accuracy=0.59, loss=1.94, step=6000] 
Valid: 100% 6912/6944 [00:00<00:00, 8089.11 uttr/s, accuracy=0.50, loss=2.16]
Train: 100% 2000/2000 [00:21<00:00, 92.29 step/s, accuracy=0.55, loss=1.85, step=8000] 
Valid: 100% 6912/6944 [00:01<00:00, 6431.56 uttr/s, accuracy=0.55, loss=1.93]
Train: 100% 2000/2000 [00:22<00:00, 89.91 step/s, accuracy=0.68, loss=1.35, step=1e+4] 
Valid: 100% 6912/6944 [00:00<00:00, 11291.96 uttr/s, accuracy=0.58, loss=1.83]
                                      
Train:   0% 0/2000 [00:00<?, ? step/s]
Train:   1% 19/2000 [00:00<00:34, 57.54 step/s, accuracy=0.67, loss=1.44, step=1e+4]

Step 10000, best model saved. (accuracy=0.5755)


...

Step 50000, best model saved. (accuracy=0.7419)


Train: 100% 2000/2000 [00:57<00:00, 34.68 step/s, accuracy=0.82, loss=0.67, step=52000]
Valid: 100% 6912/6944 [00:02<00:00, 3362.27 uttr/s, accuracy=0.74, loss=1.09]
Train: 100% 2000/2000 [00:56<00:00, 35.29 step/s, accuracy=0.82, loss=0.81, step=54000]
Valid: 100% 6912/6944 [00:02<00:00, 2901.50 uttr/s, accuracy=0.75, loss=1.13]
Train: 100% 2000/2000 [00:59<00:00, 33.48 step/s, accuracy=0.84, loss=0.60, step=56000]
Valid: 100% 6912/6944 [00:01<00:00, 3522.15 uttr/s, accuracy=0.75, loss=1.10]
Train: 100% 2000/2000 [00:58<00:00, 33.90 step/s, accuracy=0.79, loss=0.76, step=58000]
Valid: 100% 6912/6944 [00:02<00:00, 2489.77 uttr/s, accuracy=0.75, loss=1.09]
Train: 100% 2000/2000 [00:58<00:00, 34.10 step/s, accuracy=0.78, loss=0.86, step=6e+4] 
Valid: 100% 6912/6944 [00:02<00:00, 3071.32 uttr/s, accuracy=0.74, loss=1.10]
                                      
Train:   0% 0/2000 [00:00<?, ? step/s]
Train:   1% 16/2000 [00:00<00:22, 88.81 step/s, accuracy=0.84, loss=0.63, step=6e+4]

Step 60000, best model saved. (accuracy=0.7533)


Train: 100% 2000/2000 [01:10<00:00, 28.34 step/s, accuracy=0.86, loss=0.57, step=62000]
Valid: 100% 6912/6944 [00:02<00:00, 3049.28 uttr/s, accuracy=0.76, loss=1.06]
Train: 100% 2000/2000 [01:07<00:00, 29.67 step/s, accuracy=0.83, loss=0.64, step=64000]
Valid: 100% 6912/6944 [00:02<00:00, 2979.80 uttr/s, accuracy=0.76, loss=1.08]
Train: 100% 2000/2000 [01:05<00:00, 30.69 step/s, accuracy=0.77, loss=0.88, step=66000]
Valid: 100% 6912/6944 [00:02<00:00, 3249.49 uttr/s, accuracy=0.75, loss=1.06]
Train: 100% 2000/2000 [00:58<00:00, 34.34 step/s, accuracy=0.85, loss=0.61, step=68000]
Valid: 100% 6912/6944 [00:02<00:00, 3242.91 uttr/s, accuracy=0.76, loss=1.05]
Train: 100% 2000/2000 [01:00<00:00, 32.98 step/s, accuracy=0.78, loss=0.69, step=7e+4] 
Valid: 100% 6912/6944 [00:01<00:00, 3513.05 uttr/s, accuracy=0.75, loss=1.10]
                                      
Train:   0% 0/2000 [00:00<?, ? step/s]
Train:   0% 0/2000 [00:00<?, ? step/s]

Step 70000, best model saved. (accuracy=0.7593)


In [16]:

import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
  def __init__(self, data_dir):
    testdata_path = Path(data_dir) / "testdata.json"
    metadata = json.load(testdata_path.open())
    self.data_dir = data_dir
    self.data = metadata["utterances"]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    utterance = self.data[index]
    feat_path = utterance["feature_path"]
    mel = torch.load(os.path.join(self.data_dir, feat_path))

    return feat_path, mel


def inference_collate_batch(batch):
  """Collate a batch of data."""
  feat_paths, mels = zip(*batch)

  return feat_paths, torch.stack(mels)

In [17]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
  """arguments"""
  config = {
    "data_dir": "../../Dataset_for_chapter4",
    "model_path": "./model.ckpt",
    "output_path": "./output.csv",
  }

  return config


def main(
  data_dir,
  model_path,
  output_path,
):
  """Main function."""
  device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  mapping_path = Path(data_dir) / "mapping.json"
  mapping = json.load(mapping_path.open())

  dataset = InferenceDataset(data_dir)
  dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=8,
    collate_fn=inference_collate_batch,
  )
  print(f"[Info]: Finish loading data!",flush = True)

  speaker_num = len(mapping["id2speaker"])
  model = Classifier(n_spks=speaker_num).to(device)
  model.load_state_dict(torch.load(model_path))
  model.eval()
  print(f"[Info]: Finish creating model!",flush = True)

  results = [["Id", "Category"]]
  for feat_paths, mels in tqdm(dataloader):
    with torch.no_grad():
      mels = mels.to(device)
      outs = model(mels)
      preds = outs.argmax(1).cpu().numpy()
      for feat_path, pred in zip(feat_paths, preds):
        results.append([feat_path, mapping["id2speaker"][str(pred)]])

  # with open(output_path, 'w', newline='') as csvfile:
  #   writer = csv.writer(csvfile)
  #   writer.writerows(results)


if __name__ == "__main__":
  main(**parse_args())

[Info]: Use cuda:3 now!
[Info]: Finish loading data!
[Info]: Finish creating model!
